In [6]:
import os

# Set the Spark version
spark_version = '3.5.1'
os.environ['SPARK_VERSION'] = spark_version

# Install Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null

# Download and install Spark
!wget -q https://dlcdn.apache.org/spark/spark-3.5.1/spark-3.5.1-bin-hadoop3.tgz
!tar xf spark-3.5.1-bin-hadoop3.tgz
!pip install -q findspark

# Set environment variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/spark-3.5.1-bin-hadoop3"

# Initialize Spark session
import findspark
findspark.init()

Hit:1 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done


In [7]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [37]:
from pyspark.sql.functions import date_format

# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"

# Add file to Spark Context
spark.sparkContext.addFile(url)

# Read CSV into DataFrame
sales_data = spark.read.option("header", "true").csv(SparkFiles.get("home_sales_revised.csv"), inferSchema=True, sep=",", timestampFormat="m/d/yyyy")

# Convert date column to desired format
sales_data = sales_data.withColumn("date", date_format("date", "MM/dd/yyyy"))

# Show DataFrame
sales_data.show()

+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|04/08/2022|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|06/13/2021|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|04/12/2019|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|10/16/2019|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|01/08/2022|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
|5aa00529-0533-46b...|01/30/2019|      2017|218712|       2|        3|       196

In [38]:
# 2. Create a temporary view of the DataFrame.

sales_data.createOrReplaceTempView('sales')

In [39]:
sales_data.printSchema()

root
 |-- id: string (nullable = true)
 |-- date: string (nullable = true)
 |-- date_built: integer (nullable = true)
 |-- price: integer (nullable = true)
 |-- bedrooms: integer (nullable = true)
 |-- bathrooms: integer (nullable = true)
 |-- sqft_living: integer (nullable = true)
 |-- sqft_lot: integer (nullable = true)
 |-- floors: integer (nullable = true)
 |-- waterfront: integer (nullable = true)
 |-- view: integer (nullable = true)



In [41]:
# 3. What is the average price for a four bedroom house sold per year, rounded to two decimal places?
result = spark.sql("""
    SELECT
        date_built,
        ROUND(AVG(price), 2) AS average_price_4bedroom
    FROM
        sales
    WHERE
        bedrooms = 4
    GROUP BY
        date_built
""")

result.show()

+----------+----------------------+
|date_built|average_price_4bedroom|
+----------+----------------------+
|      2015|             307908.86|
|      2013|             299999.39|
|      2014|             299073.89|
|      2012|             298233.42|
|      2016|             296050.24|
|      2010|             296800.75|
|      2011|              302141.9|
|      2017|             296576.69|
+----------+----------------------+



In [42]:
# 4. What is the average price of a home for each year the home was built,
# that have 3 bedrooms and 3 bathrooms, rounded to two decimal places?
result = spark.sql("""
    SELECT
        date_built,
        ROUND(AVG(price), 2) AS average_price_3bed3bath
    FROM
        sales
    WHERE
        bedrooms = 3 AND bathrooms =3
    GROUP BY
        date_built
""")

result.show()

+----------+-----------------------+
|date_built|average_price_3bed3bath|
+----------+-----------------------+
|      2015|               288770.3|
|      2013|              295962.27|
|      2014|              290852.27|
|      2012|              293683.19|
|      2016|              290555.07|
|      2010|              292859.62|
|      2011|              291117.47|
|      2017|              292676.79|
+----------+-----------------------+



In [45]:
# 5. What is the average price of a home for each year the home was built,
# that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet, rounded to two decimal places?
result = spark.sql("""
    SELECT
        date_built,
        ROUND(AVG(price), 2) AS average_price_3bed3bath2f2kft
    FROM
        sales
    WHERE
        bedrooms = 3 AND bathrooms = 3 AND floors = 2 AND sqft_living >= 2000
    GROUP BY
        date_built
""")

result.show()


+----------+-----------------------------+
|date_built|average_price_3bed3bath2f2kft|
+----------+-----------------------------+
|      2015|                    297609.97|
|      2013|                    303676.79|
|      2014|                    298264.72|
|      2012|                    307539.97|
|      2016|                     293965.1|
|      2010|                    285010.22|
|      2011|                    276553.81|
|      2017|                    280317.58|
+----------+-----------------------------+



In [48]:
# 6. What is the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000? Order by descending view rating.
# Although this is a small dataset, determine the run time for this query.

import time

start_time = time.time()

result = spark.sql("""
    SELECT
        view,
        ROUND(AVG(price), 2) AS average_price_viewtype350k
    FROM
        sales
    GROUP BY
        view
    HAVING
        AVG(price) >= 350000
    ORDER BY
        view DESC
""")
result.show()
print("--- %s seconds ---" % (time.time() - start_time))

+----+--------------------------+
|view|average_price_viewtype350k|
+----+--------------------------+
| 100|                 1026669.5|
|  99|                1061201.42|
|  98|                1053739.33|
|  97|                1129040.15|
|  96|                1017815.92|
|  95|                 1054325.6|
|  94|                 1033536.2|
|  93|                1026006.06|
|  92|                 970402.55|
|  91|                1137372.73|
|  90|                1062654.16|
|  89|                1107839.15|
|  88|                1031719.35|
|  87|                 1072285.2|
|  86|                1070444.25|
|  85|                1056336.74|
|  84|                1117233.13|
|  83|                1033965.93|
|  82|                 1063498.0|
|  81|                1053472.79|
+----+--------------------------+
only showing top 20 rows

--- 0.9247815608978271 seconds ---


In [49]:
# 7. Cache the the temporary table home_sales.
spark.table("sales").cache()

DataFrame[id: string, date: string, date_built: int, price: int, bedrooms: int, bathrooms: int, sqft_living: int, sqft_lot: int, floors: int, waterfront: int, view: int]

In [50]:
# 8. Check if the table is cached.
spark.catalog.isCached('sales')

True

In [51]:
# 9. Using the cached data, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the uncached runtime.

start_time = time.time()

# Execute the query using the cached data
result_cached = spark.sql("""
    SELECT
        view,
        ROUND(AVG(price), 2) AS average_price_viewtype350k
    FROM
        sales
    GROUP BY
        view
    HAVING
        AVG(price) >= 350000
    ORDER BY
        view DESC
""")

# End time measurement
end_time = time.time()

#Show Result
result_cached.show()

print("--- %s seconds ---" % (time.time() - start_time))


+----+--------------------------+
|view|average_price_viewtype350k|
+----+--------------------------+
| 100|                 1026669.5|
|  99|                1061201.42|
|  98|                1053739.33|
|  97|                1129040.15|
|  96|                1017815.92|
|  95|                 1054325.6|
|  94|                 1033536.2|
|  93|                1026006.06|
|  92|                 970402.55|
|  91|                1137372.73|
|  90|                1062654.16|
|  89|                1107839.15|
|  88|                1031719.35|
|  87|                 1072285.2|
|  86|                1070444.25|
|  85|                1056336.74|
|  84|                1117233.13|
|  83|                1033965.93|
|  82|                 1063498.0|
|  81|                1053472.79|
+----+--------------------------+
only showing top 20 rows

--- 2.580533266067505 seconds ---


In [62]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data

# Read the Parquet data
parquet_sales_data = spark.read.parquet("parquet_sales")

# Partition the data by the "date_built" field
partitioned_path = "partitioned_parquet_sales"
parquet_sales_data.write.partitionBy("date_built").parquet(partitioned_path, mode="overwrite")


In [63]:
# 11. Read the parquet formatted data.
# Read the partitioned Parquet data
partitioned_parquet_sales_data = spark.read.parquet("partitioned_parquet_sales")

# Show the DataFrame
partitioned_parquet_sales_data.show()

+--------------------+----------+------+--------+---------+-----------+--------+------+----------+----+----------+
|                  id|      date| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|date_built|
+--------------------+----------+------+--------+---------+-----------+--------+------+----------+----+----------+
|2ed8d509-7372-46d...|08/06/2021|258710|       3|        3|       1918|    9666|     1|         0|  25|      2015|
|941bad30-eb49-4a7...|05/09/2020|229896|       3|        3|       2197|    8641|     1|         0|   3|      2015|
|c797ca12-52cd-4b1...|06/08/2019|288650|       2|        3|       2100|   10419|     2|         0|   7|      2015|
|0cfe57f3-28c2-472...|10/04/2019|308313|       3|        3|       1960|    9453|     2|         0|   2|      2015|
|d715f295-2fbf-4e9...|05/17/2021|391574|       3|        2|       1635|    8040|     2|         0|  10|      2015|
|a18515a2-86f3-46b...|02/18/2022|419543|       3|        2|       1642|   12826|

In [64]:
# 12. Create a temporary table for the parquet data.
partitioned_parquet_sales_data.createOrReplaceTempView("parquet_sales_data")

In [65]:
# 13. Using the parquet DataFrame, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the cached runtime.

start_time = time.time()

# Execute the query using the cached data
result_parquet = spark.sql("""
    SELECT
        view,
        ROUND(AVG(price), 2) AS average_price_viewtype350k
    FROM
        parquet_sales_data
    GROUP BY
        view
    HAVING
        AVG(price) >= 350000
    ORDER BY
        view DESC
""")

# End time measurement
end_time = time.time()

#Show Result
result_parquet.show()

print("--- %s seconds ---" % (time.time() - start_time))


+----+--------------------------+
|view|average_price_viewtype350k|
+----+--------------------------+
| 100|                 1026669.5|
|  99|                1061201.42|
|  98|                1053739.33|
|  97|                1129040.15|
|  96|                1017815.92|
|  95|                 1054325.6|
|  94|                 1033536.2|
|  93|                1026006.06|
|  92|                 970402.55|
|  91|                1137372.73|
|  90|                1062654.16|
|  89|                1107839.15|
|  88|                1031719.35|
|  87|                 1072285.2|
|  86|                1070444.25|
|  85|                1056336.74|
|  84|                1117233.13|
|  83|                1033965.93|
|  82|                 1063498.0|
|  81|                1053472.79|
+----+--------------------------+
only showing top 20 rows

--- 0.7990694046020508 seconds ---


In [58]:
# 14. Uncache the home_sales temporary table.
spark.table("sales").unpersist()

DataFrame[id: string, date: string, date_built: int, price: int, bedrooms: int, bathrooms: int, sqft_living: int, sqft_lot: int, floors: int, waterfront: int, view: int]

In [60]:
# 15. Check if the home_sales is no longer cached
is_cached = spark.table("sales").is_cached
print("Is sales cached?", is_cached)


Is sales cached? False
